

# HeatBug Modeling Approach Overview  

## Core Model Elements  
1. **Heat Field Grid (HeatGrid)**  
   - 2D temperature field governed by discrete diffusion equation:  
     $$ T_{t+1}(x,y) = (1-\alpha)T_t(x,y) + \beta \nabla^2 T $$  
     where $\alpha$ = decay coefficient, $\beta$ = diffusion coefficient  
   - Moore neighborhood used for Laplacian operator $\nabla^2 T$ computation  
   - Supports dynamic temperature field visualization  

2. **Agent Properties (HeatBug)**  
   - Thermal preference parameter: $T_{ideal} \in [20,40]$  
   - Movement energy cost model: $\Delta E = k \cdot ||\vec{p}_t - \vec{p}_{t-1}||^2$  
   - Perception range: Moore neighborhood (3x3 grid)  
   - Decision logic: Gradient descent to locate thermally optimal zones  

3. **Emergent Group Behavior**  
   - Heat field-agent positive feedback mechanism:  
     - High-density clusters generate heat accumulation  
     - Thermal distribution changes guide group migration  
   - Parameter sensitivity:  
     - $\alpha$ controls heat field memory effect  
     - $\beta$ governs heat propagation speed  

## Functions to Implement  
| Phase  | Class/Method                     | Implementation Description                          |  
|--------|----------------------------------|-----------------------------------------------------|  
| Phase1 | `HeatGrid.diffuse_heat()`        | Discrete heat diffusion computation based on Moore neighborhood |  
|        | `HeatBug.sense_environment()`    | Collect 3x3 local temperature matrix                |  
|        | `HeatBug.move()`                 | Energy consumption calculation based on squared displacement |  
|        | `HeatBug.decide_movement()`      | Temperature gradient-driven directional decision algorithm |  
| **Phase2** | **ThermalValidation.run()**      | Validate thermal diffusion model correctness including discrete Laplacian computation and parameter analysis |  
|        | **BehaviorValidation.run()**     | Verify movement logic with energy conservation and boundary constraints |  
|        | **CollectiveDistribution.analyze()** | Analyze distribution differences among groups with varying temperature preferences and parameter impacts |  

#### References
- [Heatbugs NetLogo Models Library documentation](https://ccl.northwestern.edu/netlogo/models/Heatbugs)
- [NetLogo Center for Connected Learning and Computer-Based Modeling](https://ccl.northwestern.edu/netlogo/)




# Phase 1: Building the basic framework

In [ ]:
import numpy as np
from scipy.signal import convolve2d

class HeatGrid:
    def __init__(self, size=50, alpha=0.1, beta=0.5):
        self.grid = np.zeros((size, size))
        self.alpha = alpha  # Decay coefficient
        self.beta = beta    # Diffusion coefficient

    def diffuse_heat(self):
        """Implements the heat conduction equation"""
        kernel = np.array([[0.05, 0.2, 0.05],
                          [0.2, -1.0, 0.2],
                          [0.05, 0.2, 0.05]])  # Laplacian operator convolution kernel
        laplacian = convolve2d(self.grid, kernel, mode='same')
        new_grid = (1-self.alpha)*self.grid + self.beta*laplacian
        self.grid = np.clip(new_grid, 0, 100)  # Temperature clamping

class HeatBug:
    def __init__(self, x, y, ideal_temp=35, k=0.1, grid_size=50):
        self.x = x
        self.y = y
        self.energy = 100
        self.ideal_temp = np.clip(ideal_temp, 20, 40)
        self.k = k  # Energy consumption coefficient
        self.grid_size = grid_size

    def sense_environment(self, grid):
        """Get 3x3 neighborhood temperature (with circular boundary handling)"""
        pad_grid = np.pad(grid, 1, mode='wrap')
        return pad_grid[self.x:self.x+3, self.y:self.y+3]

    def decide_movement(self, local_temp):
        """Optimized gradient descent decision algorithm"""
        # Calculate normalized temperature gradient
        grad_x = (local_temp[1,2] - local_temp[1,0]) / (2 + 1e-8)
        grad_y = (local_temp[2,1] - local_temp[0,1]) / (2 + 1e-8)
        grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)

        temp_diff = local_temp[1,1] - self.ideal_temp
        direction_sign = -1 if temp_diff > 0 else 1

        base_step = np.clip(abs(temp_diff)/5, 0.5, 2)
        noise = np.random.uniform(0.8, 1.2)
        step_size = base_step * noise

        if grad_magnitude < 0.1:
            new_x = self.x + np.random.choice([-1,0,1])
            new_y = self.y + np.random.choice([-1,0,1])
        else:
            norm_grad_x = grad_x / (grad_magnitude + 1e-8)
            norm_grad_y = grad_y / (grad_magnitude + 1e-8)

            delta_x = direction_sign * norm_grad_x * step_size
            delta_y = direction_sign * norm_grad_y * step_size

            # Limit step size to 1 grid cell
            delta_x = np.clip(delta_x, -1, 1)
            delta_y = np.clip(delta_y, -1, 1)

            new_x = int(round(self.x + delta_x)) % self.grid_size
            new_y = int(round(self.y + delta_y)) % self.grid_size

        return new_x, new_y

    def move(self, new_x, new_y):
        """Movement method with boundary constraints"""
        new_x = np.clip(new_x, 0, self.grid_size-1)
        new_y = np.clip(new_y, 0, self.grid_size-1)
        dx = new_x - self.x
        dy = new_y - self.y
        self.energy -= self.k * (dx**2 + dy**2)
        self.x, self.y = new_x, new_y


In [ ]:
# Test case 1: Strong gradient environment
local_temp = np.array([[20, 25, 30],
                      [35, 40, 45],
                      [50, 55, 60]])
# Expected movement direction: move towards cooler area (opposite to gradient direction)
bug = HeatBug(1,1, ideal_temp=20)
assert bug.decide_movement(local_temp) == (0,0)

# Test case 2: Weak gradient with random exploration
local_temp = np.ones((3,3))*30 + np.random.normal(0,0.05,size=(3,3))
bug = HeatBug(25,25, ideal_temp=30)
# Should trigger random movement (any value in [-1,0,1])
print(bug.decide_movement(local_temp))  # May output like (24,26)

(np.int64(25), np.int64(26))


# Phase 2: Understanding ABM-heatbug


## Task 1: Thermal Dynamics Validation
**Implementation Objectives**  
Validate correctness of heat diffusion model with requirements:  
- Implement discrete Laplacian computation  
- Observe impacts of different diffusion parameters (α, β)  
- Ensure temperature values remain stable in [0,100] range  

**Code Validation Tips**  

In [ ]:
# Heat diffusion process validation
test_grid = HeatGrid(alpha=0.2, beta=0.6)
test_grid.grid[24:26,24:26] = 50  # Central heat source initialization

# Perform three diffusion rounds and visualize
for step in range(3):
    test_grid.diffuse_heat()
    plt.imshow(test_grid.grid, cmap='inferno')
    plt.title(f'Step {step}')  # Display current step number
    plt.colorbar()  # Show temperature scale
    plt.show()  # Render visualization


## Task 2: Agent Behavior Validation
**Implementation Objectives**  
Validate basic movement logic with requirements:  
- Implement energy consumption based on movement distance  
- Enforce grid boundary constraints  
- Maintain energy conservation principle  


**Code Tips**  

In [ ]:
    """Movement method with boundary detection"""
    dx = new_x - self.x
    dy = new_y - self.y

    # Calculate energy consumption (squared displacement)
    self.energy -= self.k * (dx**2 + dy**2)

    # Boundary constraints
    self.x = np.clip(new_x, 0, self.grid_size-1)
    self.y = np.clip(new_y, 0, self.grid_size-1)

## Task 3: Population Distribution Observation
**Implementation Objectives**  
Observe thermal preference-induced distribution variations with requirements:  
- Implement dual populations with divergent thermal preferences  
- Establish agent-environment thermal interaction mechanisms  
- Interpret parameter impacts on distribution patterns  

**Code Tips**  

In [ ]:
# Dual population initialization
warm_group = [HeatBug(ideal_temp=38) for _ in range(20)]
cool_group = [HeatBug(ideal_temp=22) for _ in range(20)]

# Coupled simulation execution
for _ in range(100):
    # Agents release heat to environment
    for bug in warm_group + cool_group:
        bug.release_heat(grid)  # Requires heat release method implementation

    grid.diffuse_heat()

# Final state visualization
plt.scatter([b.x for b in warm_group], [b.y for b in warm_group], c='r', label='Warm')
plt.scatter([b.x for b in cool_group], [b.y for b in cool_group], c='b', label='Cool')
plt.imshow(grid.grid, cmap='inferno', alpha=0.5)
plt.legend()
